In [1]:
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
def read_data(drunkfile, normalfile):
    drunk = pd.read_csv(drunkfile)
    normal = pd.read_csv(normalfile)
    data = pd.concat([drunk, normal])
    ans_list = [1]*len(drunk) + [0]*len(normal)
    return data[['smile','anger','contempt','disgust','fear','happiness','neutral','sadness','surprise','eyeOccluded']].values, ans_list

In [3]:
X_train, Y_train = read_data("drunk.csv", "normal.csv")

In [4]:
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [5]:
X_train[X_train==True] = 1
X_train[X_train==False] = 0

In [6]:
(X_train, Y_train) = shuffle(X_train, Y_train)

In [7]:
X_train, X_valid = X_train[:-50], X_train[-50:]
Y_train, Y_valid = Y_train[:-50], Y_train[-50:]

In [8]:
model = Sequential()
model.add(Dense(input_dim = X_train.shape[1], units = 10, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 8, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
# callbacks = []
# callbacks.append(ModelCheckpoint('models/model-{epoch:05d}-{val_acc:.5f}-{val_loss:.5f}.npy', monitor='val_acc', save_best_only=True, mode = 'auto', period=1))
# model.fit(X_train, Y_train, batch_size = 8, epochs = 100, validation_split=0.2, callbacks=callbacks)
model.fit(X_train, Y_train, batch_size = 8, epochs = 100, validation_split=0.2)
result = model.evaluate(X_valid, Y_valid, batch_size = X_valid.shape[0])
print(result[1])

Train on 1018 samples, validate on 255 samples
Epoch 1/100
1018/1018 [==============================] - 2s - loss: 0.6363 - acc: 0.6601 - val_loss: 0.6076 - val_acc: 0.6431
Epoch 2/100
1018/1018 [==============================] - 0s - loss: 0.6122 - acc: 0.6582 - val_loss: 0.5944 - val_acc: 0.6510
Epoch 3/100
1018/1018 [==============================] - 0s - loss: 0.6113 - acc: 0.6719 - val_loss: 0.5868 - val_acc: 0.6784
Epoch 4/100
1018/1018 [==============================] - 0s - loss: 0.6082 - acc: 0.6945 - val_loss: 0.5778 - val_acc: 0.7098
Epoch 5/100
1018/1018 [==============================] - 0s - loss: 0.6057 - acc: 0.6916 - val_loss: 0.5680 - val_acc: 0.7255
Epoch 6/100
1018/1018 [==============================] - 0s - loss: 0.5923 - acc: 0.6955 - val_loss: 0.5617 - val_acc: 0.7333
Epoch 7/100
1018/1018 [==============================] - 0s - loss: 0.5925 - acc: 0.7063 - val_loss: 0.5435 - val_acc: 0.7608
Epoch 8/100
1018/1018 [==============================] - 0s - loss: 0.5

1018/1018 [==============================] - 0s - loss: 0.4810 - acc: 0.7937 - val_loss: 0.4481 - val_acc: 0.8196
Epoch 66/100
1018/1018 [==============================] - 0s - loss: 0.4663 - acc: 0.8065 - val_loss: 0.4429 - val_acc: 0.8314
Epoch 67/100
1018/1018 [==============================] - 0s - loss: 0.4843 - acc: 0.7908 - val_loss: 0.4468 - val_acc: 0.8275
Epoch 68/100
1018/1018 [==============================] - 0s - loss: 0.4745 - acc: 0.7947 - val_loss: 0.4437 - val_acc: 0.8314
Epoch 69/100
1018/1018 [==============================] - 0s - loss: 0.4753 - acc: 0.7986 - val_loss: 0.4422 - val_acc: 0.8314
Epoch 70/100
1018/1018 [==============================] - 0s - loss: 0.4900 - acc: 0.7839 - val_loss: 0.4517 - val_acc: 0.8196
Epoch 71/100
1018/1018 [==============================] - 0s - loss: 0.4723 - acc: 0.8026 - val_loss: 0.4471 - val_acc: 0.8392
Epoch 72/100
1018/1018 [==============================] - 0s - loss: 0.4766 - acc: 0.8055 - val_loss: 0.4482 - val_acc: 0.81

In [9]:
from xgboost import XGBClassifier
model2 = XGBClassifier(max_depth=3, n_estimators=200, learning_rate=0.05)
model2.fit(X_train, Y_train.flatten())
predict_train = model2.predict(X_train)
acc_train = np.mean(Y_train.flatten() == predict_train)
print("acc_train = ", acc_train)
predict_valid = model2.predict(X_valid)
acc_valid = np.mean(Y_valid.flatten() == predict_valid)
print("acc_valid = ", acc_valid)

acc_train =  0.862529457973
acc_valid =  0.82


c:\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
from keras.models import load_model
model = load_model("keras_model.h5")
result = model.evaluate(X_valid, Y_valid, batch_size = X_valid.shape[0])
print(result[1])

50/50 [==============================] - 0s
0.839999973774
